# Usage Example

In [1]:
import os
from birdy import WPSClient
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
# url = get_target_url("chickadee")
# print(f"Using chickadee on {url}")

In [3]:
chickadee = WPSClient("http://localhost:5004/wps")

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f6b1cd94ef0>
File:            ~/.local/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for PCIC's ClimDown package.

Processes
---------

hello
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.

ci
    Climate Imprint (CI) downscaling
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS provider. config (Config): an instance
    processes: Specify a subset of processes to bind. Defaults to all
        processes.
    converters (dict): Correspondence of {mimetype: class} to convert
        this mimetype to a python object.
    username (str): passed to :class:`owslib.wps.WebProcessingService`
    password (str):

In [5]:
chickadee.ci(
    gcm_file = resource_filename("tests", "/data/tiny_gcm.nc"),
    obs_file = resource_filename("tests", "/data/tiny_obs.nc"),
    varname = "tasmax",
)

 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process failed, please check server error log'}
